# Importing Relevant Libraries

In [1]:
!pip install torchsummary
!pip install watermark

In [2]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import torch
from torchsummary import summary
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torchvision.models as models 
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import ConcatDataset
from torch.utils.tensorboard import SummaryWriter

In [3]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [4]:
from watermark import watermark

# Printing versions of libraries used
%load_ext watermark
%watermark -v -p torch,torchvision,torchsummary,numpy,matplotlib,PIL

Python implementation: CPython
Python version       : 3.10.14
IPython version      : 8.21.0

torch       : 2.4.0
torchvision : 0.19.0
torchsummary: 1.5.1
numpy       : 1.26.4
matplotlib  : 3.7.5
PIL         : 10.4.0



# Getting Dataset Ready

In [5]:
class AeroHeliDataset(Dataset):
    def __init__(self, root_dir, train=True, train_transform=None, test_transform=None):
        """
        Args:
            root_dir (string): Path to the dataset directory.
            train (bool): If True, loads the training data. If False, loads the test data.
            train_transform (callable, optional): Transformations for training data.
            test_transform (callable, optional): Transformations for test data.
        """
        self.root_dir = root_dir
        self.train = train
        self.train_transform = train_transform
        self.test_transform = test_transform
        
        # The list of images and labels
        self.images = []
        self.labels = []
        
        # Set directory based on whether this is training or testing data
        data_type = 'train' if self.train else 'test'
        
        # Load images and labels
        for label, class_name in enumerate(['aero', 'heli']):  # 0 for 'aero', 1 for 'heli'
            class_dir = os.path.join(root_dir, data_type, class_name)
            for filename in os.listdir(class_dir):
                if filename.endswith(".jpg"):  
                    self.images.append(os.path.join(class_dir, filename))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]
        
        # Open the image
        image = Image.open(image_path).convert("RGB")
        
        # Apply appropriate transformations
        if self.train and self.train_transform:
            image = self.train_transform(image)
        elif not self.train and self.test_transform:
            image = self.test_transform(image)
        
        return image, label


In [6]:
# Defining different transforms for different models

basic_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing images to 224x224
    transforms.ToTensor()
])

augmented_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # Augmentation
    transforms.RandomHorizontalFlip(),  
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
])

In [7]:
root_dir = "/kaggle/input/aeroplane-helicopter-image-classification/dataset_aero_heli"

In [8]:
# Training datasets for each type
train_dataset_basic = AeroHeliDataset(root_dir=root_dir, train=True, train_transform=basic_transform)

only_augmented_1 = AeroHeliDataset(root_dir=root_dir, train=True, train_transform=augmented_transform)
only_augmented_2 = AeroHeliDataset(root_dir=root_dir, train=True, train_transform=augmented_transform)
train_dataset_augmented = ConcatDataset([train_dataset_basic, only_augmented_1, only_augmented_2]) # Concatinating extra augments

# Corresponding test datasets for each type (without augmentation)
test_dataset_basic = AeroHeliDataset(root_dir=root_dir, train=False, test_transform=basic_transform)


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Defining Different Models

In [10]:
# 1 Block model

class Model1Block(nn.Module):
    def __init__(self):
        super(Model1Block, self).__init__()
        # Block 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 112 * 112, 128)
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        # Block 1
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # Flatten
        x = self.flatten(x)
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        
        return x

summary(Model1Block().to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
         MaxPool2d-2         [-1, 32, 112, 112]               0
           Flatten-3               [-1, 401408]               0
            Linear-4                  [-1, 128]      51,380,352
            Linear-5                    [-1, 1]             129
Total params: 51,381,377
Trainable params: 51,381,377
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 18.38
Params size (MB): 196.00
Estimated Total Size (MB): 214.95
----------------------------------------------------------------


In [11]:
 # 3 Block model

class Model3Block(nn.Module):
    def __init__(self):
        super(Model3Block, self).__init__()
        # Block 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Block 2
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # Block 3
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)        
        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 28 * 28, 128)  # 128 channels * 28x28 (after 3 poolings)
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        # Block 1
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # Block 2
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        # Block 3
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        # Flatten
        x = self.flatten(x)
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        
        return x
        
summary(Model3Block().to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
         MaxPool2d-2         [-1, 32, 112, 112]               0
            Conv2d-3         [-1, 64, 112, 112]          18,496
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5          [-1, 128, 56, 56]          73,856
         MaxPool2d-6          [-1, 128, 28, 28]               0
           Flatten-7               [-1, 100352]               0
            Linear-8                  [-1, 128]      12,845,184
            Linear-9                    [-1, 1]             129
Total params: 12,938,561
Trainable params: 12,938,561
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 27.56
Params size (MB): 49.36
Estimated Total Size (MB): 77.49
----------------------------------

In [12]:
# Transfer learning using VGG16  with tuning all layers (including tuning convolution layers)

class VGGModel(nn.Module):
    def __init__(self):
        super(VGGModel, self).__init__()
        # Load VGG16 model with pretrained weights, including the classifier (fully connected) layers
        self.vgg16 = models.vgg16(weights='VGG16_Weights.DEFAULT')
        
        # Freeze all convolutional (feature extraction) layers, making them non-trainable
        for param in self.vgg16.features.parameters():
            param.requires_grad = True
        
        # Modify the final fully connected layer in the classifier for binary classification
        num_features = self.vgg16.classifier[-1].in_features  # Get input features of the last layer
        self.vgg16.classifier[-1] = nn.Linear(num_features, 1)  # Replace with a layer that outputs 1 value
        
    def forward(self, x):
        # Forward pass through the entire model, including modified classifier
        x = self.vgg16(x)
        x = torch.sigmoid(x)  # Apply sigmoid to get probabilities for binary classification
        return x

summary(VGGModel().to(device), input_size=(3, 224, 224))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 216MB/s]  


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [13]:
# Transfer learning using VGG16 or VGG19 with tuning only final MLP layers (excluding convolution layers)

class VGGModel_1(nn.Module):
    def __init__(self):
        super(VGGModel_1, self).__init__()
        # Load VGG16 model with pretrained weights, including the classifier (fully connected) layers
        self.vgg16 = models.vgg16(weights='VGG16_Weights.DEFAULT')
        
        # Freeze all convolutional (feature extraction) layers, making them non-trainable
        for param in self.vgg16.features.parameters():
            param.requires_grad = False
        
        # Modify the final fully connected layer in the classifier for binary classification
        num_features = self.vgg16.classifier[-1].in_features  # Get input features of the last layer
        self.vgg16.classifier[-1] = nn.Linear(num_features, 1)  # Replace with a layer that outputs 1 value
        
    def forward(self, x):
        # Forward pass through the entire model, including modified classifier
        x = self.vgg16(x)
        x = torch.sigmoid(x)  # Apply sigmoid to get probabilities for binary classification
        return x
        
summary(VGGModel_1().to(device), [(3, 224, 224)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [14]:
def get_model(model_name):
    if model_name == "1_block":
        model = Model1Block()
    elif model_name == "3_block":
        model = Model3Block()
    elif model_name == "3_block_aug":
        model = Model3Block()
    elif model_name == "vgg_16":
        model = VGGModel()
    elif model_name == "vgg_16_1":
        model = VGGModel_1()
    else:
        raise ValueError(f"Model {model_name} is not defined")
    return model

# Training and Evaluation

In [15]:
# Hyperparameters
num_epochs = 10
batch_size = 16
learning_rate = 1e-4

In [16]:
def log_images_with_predictions(data, targets, predictions, writer, step):
    true_labels = targets.squeeze().tolist()
    pred_labels = predictions.squeeze().tolist()

    # Set up the plot grid for 16 images (since batch size is 16)
    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(16, 12))  
    axes = axes.flatten()

    for img, true_label, pred_label, ax in zip(data[:16], true_labels[:16], pred_labels[:16], axes):
        img = img.permute(1, 2, 0).cpu().numpy()

        # Plot the image
        ax.imshow(img)
        ax.axis('off')

        # Set title with true/pred labels
        title_color = "green" if true_label == pred_label else "red"
        ax.set_title(f"True: {int(true_label)} | Pred: {int(pred_label)}", color=title_color)

    # Save the plot to a temporary file
    plt.tight_layout()
    plt_path = "temp_plot_final_epoch.jpg"
    plt.savefig(plt_path, bbox_inches='tight', format='jpg')
    plt.close(fig)
    
    pil_img = Image.open(plt_path)
    pil_img = pil_img.convert("RGB")
    img_np = np.array(pil_img)

    # Log the image to TensorBoard
    writer.add_image("Test Images with Predictions", img_np.transpose(2, 0, 1), global_step=step)


In [17]:
def train_and_evaluate(model_name, train_dataset, test_dataset,num_epochs = 10, batch_size = 16, learning_rate = 1e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize model
    model = get_model(model_name)
    model.to(device)

    # Define loss and optimizer
    criterion = nn.BCELoss()  
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Data loaders
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    # TensorBoard writers
    writer_train = SummaryWriter(f"runs/{model_name}_Train")
    writer_test = SummaryWriter(f"runs/{model_name}_Test")

    # Log model graph for first iteration
    images, _ = next(iter(train_loader))
    writer_train.add_graph(model, images.to(device))
    
    step_train, step_test = 0, 0
    total_training_time = 0

    # Initialize lists to accumulate losses and accuracies
    train_losses = []
    train_accuracies = []
    test_accuracies = []

    model_dir = "/kaggle/working/saved_models"
    os.makedirs(model_dir, exist_ok=True)

    # Training and evaluation loop
    for epoch in range(num_epochs):
        # Training phase
        start_time = time.time()
        model.train()
        for batch_idx, (data, targets) in enumerate(train_loader):
            data = torch.tensor(data, dtype=torch.float32).to(device)
            targets = torch.tensor(targets, dtype=torch.float32).unsqueeze(1).to(device)
            
            # Forward pass
            scores = model(data)
            loss = criterion(scores, targets)
            train_losses.append(loss.item())
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Calculate training accuracy
            predictions = (scores > 0.5).float()  
            correct = (predictions == targets).sum().item()
            accuracy = correct / data.shape[0]
            train_accuracies.append(accuracy)

            # Log to TensorBoard for training
            writer_train.add_scalar("Loss", loss.item(), global_step=step_train)
            writer_train.add_scalar("Accuracy", accuracy, global_step=step_train)
            
            step_train += 1

        # End timing the training phase
        end_time = time.time()
        total_training_time += end_time - start_time
        
        # Test phase (no gradients needed)
        model.eval()
        with torch.no_grad():
            for data, targets in test_loader:
                data = torch.tensor(data, dtype=torch.float32).to(device)
                targets = torch.tensor(targets, dtype=torch.float32).unsqueeze(1).to(device)
                
                # Forward pass
                scores = model(data)
                loss = criterion(scores, targets)

                # Calculate test accuracy
                predictions = (scores > 0.5).float() 
                correct = (predictions == targets).sum().item()
                accuracy = correct / data.shape[0]
                test_accuracies.append(accuracy)

                # Log to TensorBoard for testing
                writer_test.add_scalar("Accuracy", accuracy, global_step=step_test)
                
                # Accumulate final predictions to visualize
                if epoch == num_epochs - 1:
                    log_images_with_predictions(data, targets, predictions, writer_test, step_test)
                
                step_test += 1
                
        # Calculate and print the epoch metrics
        avg_train_loss = sum(train_losses) / len(train_losses)
        avg_train_acc = sum(train_accuracies) / len(train_accuracies)
        avg_test_acc = sum(test_accuracies) / len(test_accuracies)

        print(f"\nEpoch [{epoch+1}/{num_epochs}] - "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Train Accuracy: {avg_train_acc:.4f}, "
              f"Test Accuracy: {avg_test_acc:.4f}, ")

    # Save final model
    torch.save(model.state_dict(), os.path.join(model_dir, f"{model_name}_final.pth"))
    
    print(f"\n------------\nTotal Training Time: {total_training_time:.4f} seconds \n-------------\n")  
    print(f"\n------------\nPer Epoch Training Time: {(total_training_time / num_epochs):.4f} seconds per epoch \n-------------\n")
    
    # Close the writers
    writer_train.close()
    writer_test.close()

In [18]:
train_and_evaluate(model_name="1_block", train_dataset=train_dataset_basic, test_dataset=test_dataset_basic)


Epoch [1/10] - Train Loss: 0.8571, Train Accuracy: 0.5813, Test Accuracy: 0.8750, 

Epoch [2/10] - Train Loss: 0.5695, Train Accuracy: 0.7250, Test Accuracy: 0.8542, 

Epoch [3/10] - Train Loss: 0.4488, Train Accuracy: 0.7812, Test Accuracy: 0.8750, 

Epoch [4/10] - Train Loss: 0.3729, Train Accuracy: 0.8172, Test Accuracy: 0.8802, 

Epoch [5/10] - Train Loss: 0.3177, Train Accuracy: 0.8512, Test Accuracy: 0.8917, 

Epoch [6/10] - Train Loss: 0.2812, Train Accuracy: 0.8729, Test Accuracy: 0.8958, 

Epoch [7/10] - Train Loss: 0.2539, Train Accuracy: 0.8893, Test Accuracy: 0.9048, 

Epoch [8/10] - Train Loss: 0.2288, Train Accuracy: 0.9008, Test Accuracy: 0.8984, 

Epoch [9/10] - Train Loss: 0.2084, Train Accuracy: 0.9111, Test Accuracy: 0.9005, 

Epoch [10/10] - Train Loss: 0.1904, Train Accuracy: 0.9194, Test Accuracy: 0.9042, 

------------
Total Training Time: 89.6671 seconds 
-------------


------------
Per Epoch Training Time: 8.9667 seconds per epoch 
-------------



In [19]:
train_and_evaluate(model_name="3_block", train_dataset=train_dataset_basic, test_dataset=test_dataset_basic)


Epoch [1/10] - Train Loss: 0.6801, Train Accuracy: 0.5563, Test Accuracy: 0.7500, 

Epoch [2/10] - Train Loss: 0.5906, Train Accuracy: 0.7000, Test Accuracy: 0.7917, 

Epoch [3/10] - Train Loss: 0.5242, Train Accuracy: 0.7396, Test Accuracy: 0.7986, 

Epoch [4/10] - Train Loss: 0.4788, Train Accuracy: 0.7734, Test Accuracy: 0.8073, 

Epoch [5/10] - Train Loss: 0.4398, Train Accuracy: 0.7925, Test Accuracy: 0.8250, 

Epoch [6/10] - Train Loss: 0.4042, Train Accuracy: 0.8156, Test Accuracy: 0.8368, 

Epoch [7/10] - Train Loss: 0.3729, Train Accuracy: 0.8330, Test Accuracy: 0.8482, 

Epoch [8/10] - Train Loss: 0.3428, Train Accuracy: 0.8500, Test Accuracy: 0.8542, 

Epoch [9/10] - Train Loss: 0.3149, Train Accuracy: 0.8653, Test Accuracy: 0.8634, 

Epoch [10/10] - Train Loss: 0.2911, Train Accuracy: 0.8775, Test Accuracy: 0.8729, 

------------
Total Training Time: 88.6417 seconds 
-------------


------------
Per Epoch Training Time: 8.8642 seconds per epoch 
-------------



In [20]:
train_and_evaluate(model_name="3_block_aug", train_dataset=train_dataset_augmented, test_dataset=test_dataset_basic)


Epoch [1/10] - Train Loss: 0.5962, Train Accuracy: 0.6792, Test Accuracy: 0.7917, 

Epoch [2/10] - Train Loss: 0.4877, Train Accuracy: 0.7531, Test Accuracy: 0.8646, 

Epoch [3/10] - Train Loss: 0.3870, Train Accuracy: 0.8146, Test Accuracy: 0.8889, 

Epoch [4/10] - Train Loss: 0.3164, Train Accuracy: 0.8536, Test Accuracy: 0.9062, 

Epoch [5/10] - Train Loss: 0.2662, Train Accuracy: 0.8796, Test Accuracy: 0.9167, 

Epoch [6/10] - Train Loss: 0.2270, Train Accuracy: 0.8990, Test Accuracy: 0.9236, 

Epoch [7/10] - Train Loss: 0.1979, Train Accuracy: 0.9131, Test Accuracy: 0.9286, 

Epoch [8/10] - Train Loss: 0.1757, Train Accuracy: 0.9232, Test Accuracy: 0.9323, 

Epoch [9/10] - Train Loss: 0.1578, Train Accuracy: 0.9317, Test Accuracy: 0.9352, 

Epoch [10/10] - Train Loss: 0.1427, Train Accuracy: 0.9385, Test Accuracy: 0.9375, 

------------
Total Training Time: 282.8568 seconds 
-------------


------------
Per Epoch Training Time: 28.2857 seconds per epoch 
-------------



In [21]:
train_and_evaluate(model_name="vgg_16", train_dataset=train_dataset_basic, test_dataset=test_dataset_basic, num_epochs = 5)


Epoch [1/5] - Train Loss: 0.2057, Train Accuracy: 0.9000, Test Accuracy: 0.9792, 

Epoch [2/5] - Train Loss: 0.1434, Train Accuracy: 0.9437, Test Accuracy: 0.9583, 

Epoch [3/5] - Train Loss: 0.1669, Train Accuracy: 0.9583, Test Accuracy: 0.9444, 

Epoch [4/5] - Train Loss: 0.1298, Train Accuracy: 0.9672, Test Accuracy: 0.9583, 

Epoch [5/5] - Train Loss: 0.1039, Train Accuracy: 0.9738, Test Accuracy: 0.9667, 

------------
Total Training Time: 54.7540 seconds 
-------------


------------
Per Epoch Training Time: 10.9508 seconds per epoch 
-------------



In [22]:
train_and_evaluate(model_name="vgg_16_1", train_dataset=train_dataset_basic, test_dataset=test_dataset_basic, num_epochs =  5)


Epoch [1/5] - Train Loss: 0.2532, Train Accuracy: 0.8750, Test Accuracy: 0.9792, 

Epoch [2/5] - Train Loss: 0.1288, Train Accuracy: 0.9375, Test Accuracy: 0.9792, 

Epoch [3/5] - Train Loss: 0.0860, Train Accuracy: 0.9583, Test Accuracy: 0.9792, 

Epoch [4/5] - Train Loss: 0.0645, Train Accuracy: 0.9688, Test Accuracy: 0.9792, 

Epoch [5/5] - Train Loss: 0.0516, Train Accuracy: 0.9750, Test Accuracy: 0.9792, 

------------
Total Training Time: 47.5793 seconds 
-------------


------------
Per Epoch Training Time: 9.5159 seconds per epoch 
-------------



In [23]:
!zip -r runs.zip runs

  adding: runs/ (stored 0%)
  adding: runs/vgg_16_1_Train/ (stored 0%)
  adding: runs/vgg_16_1_Train/events.out.tfevents.1731523842.c6d0534cfc01.30.8 (deflated 90%)
  adding: runs/1_block_Test/ (stored 0%)
  adding: runs/1_block_Test/events.out.tfevents.1731523151.c6d0534cfc01.30.1 (deflated 0%)
  adding: runs/3_block_Train/ (stored 0%)
  adding: runs/3_block_Train/events.out.tfevents.1731523289.c6d0534cfc01.30.2 (deflated 77%)
  adding: runs/3_block_aug_Train/ (stored 0%)
  adding: runs/3_block_aug_Train/events.out.tfevents.1731523424.c6d0534cfc01.30.4 (deflated 73%)
  adding: runs/vgg_16_1_Test/ (stored 0%)
  adding: runs/vgg_16_1_Test/events.out.tfevents.1731523842.c6d0534cfc01.30.9 (deflated 0%)
  adding: runs/3_block_aug_Test/ (stored 0%)
  adding: runs/3_block_aug_Test/events.out.tfevents.1731523424.c6d0534cfc01.30.5 (deflated 0%)
  adding: runs/vgg_16_Train/ (stored 0%)
  adding: runs/vgg_16_Train/events.out.tfevents.1731523756.c6d0534cfc01.30.6 (deflated 90%)
  adding: runs/1_b

In [24]:
!zip -r saved_models.zip saved_models

  adding: saved_models/ (stored 0%)
  adding: saved_models/3_block_final.pth (deflated 8%)
  adding: saved_models/1_block_final.pth (deflated 8%)
  adding: saved_models/3_block_aug_final.pth (deflated 7%)
  adding: saved_models/vgg_16_final.pth (deflated 7%)
  adding: saved_models/vgg_16_1_final.pth (deflated 7%)
